
# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from pprint import pprint


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath = os.path.join('..','WeatherPy','output_data','cities.csv')
df = pd.read_csv(csvpath)
df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Iskateley,95,RU,1619799923,85,67.6803,53.1512,30.13,1.45
1,1,Puducherry,16,IN,1619799924,75,11.9300,79.8300,84.54,11.30
2,2,Ushuaia,75,AR,1619799925,87,-54.8000,-68.3000,41.00,2.30
3,3,Albany,100,US,1619799654,72,42.6001,-73.9662,55.99,1.99
4,4,Boralday,90,KZ,1619799925,40,43.3578,76.8594,73.40,17.90
...,...,...,...,...,...,...,...,...,...,...
581,581,Clovis,40,US,1619800038,28,36.8252,-119.7029,77.00,6.91
582,582,Nānākuli,1,US,1619800175,78,21.3906,-158.1547,69.80,4.61
583,583,San Rafael,41,AR,1619800044,35,-34.6177,-68.3301,72.52,10.36
584,584,North Bend,1,US,1619800176,76,43.4065,-124.2243,61.00,5.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
coord = df[["Lat", "Lng"]]
# Fill NaN values and convert to float
humidity = df["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(coord, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Drop any rows will null values.

df = df.dropna()
df =df.reset_index(drop=True)

# Drop any row with cloudniness above 10%
index_value_c = df[(df['Cloudiness']>10)].index
hotel_df = df.drop(index=df.index[index_value_c])
hotel_df = hotel_df.reset_index(drop=True)
# Drop any row with max temp below 80
index_value_tmin = hotel_df[(hotel_df['Max Temp']<80)].index
hotel_df = hotel_df.drop(index=hotel_df.index[index_value_tmin])
hotel_df = hotel_df.reset_index(drop=True)
# Drop any row with max temp above 100
index_value_tmax = hotel_df[(hotel_df['Max Temp']>=100)].index
hotel_df = hotel_df.drop(index=hotel_df.index[index_value_tmax])
hotel_df = hotel_df.reset_index(drop=True)
# Drop any row with humidity above 60
index_value_hum = hotel_df[(hotel_df['Humidity']>60)].index
hotel_df = hotel_df.drop(index=hotel_df.index[index_value_hum])
hotel_df = hotel_df.reset_index(drop=True)
# Drop any row with humidity below 30
index_value_hum = hotel_df[(hotel_df['Humidity']<30)].index
hotel_df = hotel_df.drop(index=hotel_df.index[index_value_hum])
hotel_df = hotel_df.reset_index(drop=True)
# Drop any row with wind speed below 5
index_value_spmin = hotel_df[(hotel_df['Wind Speed']<5)].index
hotel_df = hotel_df.drop(index=hotel_df.index[index_value_spmin])
hotel_df = hotel_df.reset_index(drop=True)
# Drop any row with wind speed above 10
index_value_spmax = hotel_df[(hotel_df['Wind Speed']>15)].index
hotel_df = hotel_df.drop(index=hotel_df.index[index_value_spmax])

hotel_df.head(10)


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,185,Celestún,0,MX,1619800005,54,20.8667,-90.4000,87.40,9.44
1,237,Karratha,1,AU,1619800029,45,-20.7377,116.8463,82.18,6.93
2,242,Kiryat Gat,1,IL,1619800031,46,31.6100,34.7642,80.01,10.69
3,270,Sharjah city,0,AE,1619800043,45,25.3573,55.4033,86.00,6.91
4,372,Mt. Pleasant,1,US,1619799998,50,32.7941,-79.8626,84.99,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df['Hotel Name']=''
hotel_df.head(10)

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,185,Celestún,0,MX,1619800005,54,20.8667,-90.4000,87.40,9.44,
1,237,Karratha,1,AU,1619800029,45,-20.7377,116.8463,82.18,6.93,
2,242,Kiryat Gat,1,IL,1619800031,46,31.6100,34.7642,80.01,10.69,
3,270,Sharjah city,0,AE,1619800043,45,25.3573,55.4033,86.00,6.91,
4,372,Mt. Pleasant,1,US,1619799998,50,32.7941,-79.8626,84.99,6.91,


In [9]:
# geocoordinates

for i, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Lng']
    target_coordinates = f'{lat},{long}'
    target_radius = 5000
    target_type = 'lodging'

    # base url
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    params = {
        'location': target_coordinates,
        'radius' : target_radius,
        'type' : target_type,
        'key': g_key
    }
    

    #response
    try:
        response = requests.get(base_url,params=params).json()
       
        hotel_df.loc[i,['Hotel Name']] = response["results"][0]["name"]
        
   
    except:
        print('Missing field...')

               
hotel_df.head(10)

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,185,Celestún,0,MX,1619800005,54,20.8667,-90.4000,87.40,9.44,Posada Lilia
1,237,Karratha,1,AU,1619800029,45,-20.7377,116.8463,82.18,6.93,ibis Styles Karratha
2,242,Kiryat Gat,1,IL,1619800031,46,31.6100,34.7642,80.01,10.69,OlusHome
3,270,Sharjah city,0,AE,1619800043,45,25.3573,55.4033,86.00,6.91,Swiss Belhotel Sharjah
4,372,Mt. Pleasant,1,US,1619799998,50,32.7941,-79.8626,84.99,6.91,Best Western Patriots Point


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))